# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [52]:
import re
import os

In [53]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [54]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [55]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [56]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [57]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 48
HIDDEN_SIZE = 16
MAX_LEN = 50
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 50)
x_test shape: (1987, 50)
trainlabels shape: (6899, 50, 21)
testlabels shape: (1987, 50, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 48)            329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 32)            8320      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 21)            693       
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 21)            945      

6209/6209 [==============================] - 24s 4ms/step - loss: 2.9196 - crf_viterbi_accuracy: 0.9436 - val_loss: 3.0596 - val_crf_viterbi_accuracy: 0.8938
Epoch 42/50
6209/6209 [==============================] - 19s 3ms/step - loss: 2.9177 - crf_viterbi_accuracy: 0.9454 - val_loss: 3.0559 - val_crf_viterbi_accuracy: 0.8945
Epoch 43/50
6209/6209 [==============================] - 17s 3ms/step - loss: 2.9172 - crf_viterbi_accuracy: 0.9450 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8987
Epoch 44/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9157 - crf_viterbi_accuracy: 0.9464 - val_loss: 3.0587 - val_crf_viterbi_accuracy: 0.8969
Epoch 45/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9149 - crf_viterbi_accuracy: 0.9485 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8962
Epoch 46/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9139 - crf_viterbi_accuracy: 0.9489 - val_loss: 3.0592 - val_crf_viterbi_accuracy:

In [59]:
# save model
model.save("model/ch_ner_model.h5")

In [62]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体
                result_words.append((number,start,end,input_data[start: end], tag_label)) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                result_words.append((number,start,end,input_data[start: end], tag_label)) # 获取实体
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        result_words.append((number,start,end,input_data[start: end], tag_label)) # 获取结尾的实体
    return result_words

In [ ]:
maxlen = 50
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    result[i+1000] = result_words
    for res in result_words:
        number = res[0]
        start = res[1]
        end = res[2]
        word = res[3]
        tag = res[4]
        print(i+1000,'T'+str(number), start, end, "".join(word), tag)

1000 T1 138 142 清热解毒 DRUG_EFFICACY
1000 T2 143 147 化湿除带 DRUG_EFFICACY
1000 T3 148 152 祛瘀止痛 DRUG_EFFICACY
1000 T4 160 165 慢性盆腔炎 DISEASE
1001 T1 36 39 糖尿病 DISEASE
1001 T2 87 91 月经量少 SYMPTOM
1001 T3 95 101 血虚萎黄后错 SYMPTOM
1001 T4 102 106 血虚萎黄 SYMPTOM
1001 T5 107 111 风湿痹痛 SYMPTOM
1001 T7 112 116 肢体麻木 SYMPTOM
1001 T7 116 119 糖尿病 DISEASE
1002 T1 36 42 感冒发热病人 PERSON_GROUP
1002 T2 51 54 高血压 DISEASE
1002 T3 55 58 心脏病 DISEASE_GROUP
1002 T4 59 61 肝病 DISEASE_GROUP
1002 T5 66 68 肾病 DISEASE_GROUP
1002 T6 90 94 月经紊乱 SYMPTOM
1002 T7 109 111 眩晕 SYMPTOM
1002 T8 152 155 过敏者 PERSON_GROUP
1002 T9 158 163 过敏体质者 PERSON_GROUP
1002 T10 192 194 儿童 PERSON_GROUP
1002 T11 242 244 颗粒 DRUG_DOSAGE
1002 T12 245 248 气微香 DRUG_TASTE
1002 T13 249 252 味微苦 DRUG_TASTE
1002 T14 282 286 烘热汗出 SYMPTOM
1002 T15 287 291 头晕耳鸣 SYMPTOM
1002 T16 297 301 五心烦热 SYMPTOM
1002 T17 302 306 腰背酸痛 SYMPTOM
1002 T18 307 311 大便干燥 SYMPTOM
1002 T19 312 315 心烦易 SYMPTOM
1002 T21 387 389 2g SYNDROME
1002 T21 389 391 *1 SYNDROME
1003 T2 55 59 子宫肌瘤 DISEAS

1026 T1 5 7 寒凉 FOOD_GROUP
1026 T2 8 10 生冷 FOOD_GROUP
1026 T3 15 17 孕妇 PERSON_GROUP
1026 T4 18 23 糖尿病患者 PERSON_GROUP
1026 T5 35 37 感冒 DISEASE
1026 T6 86 90 月经量少 SYMPTOM
1026 T7 92 96 月经错后 SYMPTOM
1026 T8 98 105 阴道不规则出血 SYMPTOM
1026 T9 164 169 过敏体质者 PERSON_GROUP
1026 T10 284 288 补血活血 DRUG_EFFICACY
1026 T11 296 298 血虚 SYNDROME
1026 T12 299 303 月经不调 SYMPTOM
1026 T14 304 306 痛经 SYMPTOM
1026 T14 306 308 儿童 PERSON_GROUP
1026 T15 312 316 薄膜衣片 DRUG_DOSAGE
1027 T1 8 10 颗粒 DRUG_DOSAGE
1027 T2 11 13 气香 DRUG_TASTE
1027 T3 14 16 味甜 DRUG_TASTE
1027 T4 17 18 辛 DRUG_TASTE
1027 T5 21 22 辛 DRUG_TASTE
1027 T6 75 77 血虚 SYMPTOM
1027 T7 79 83 月经不调 SYMPTOM
1027 T8 84 87 痛经  SYMPTOM
1028 T1 29 31 孕妇 PERSON_GROUP
1028 T2 53 57 湿热下注 SYNDROME
1028 T3 58 62 白带过多 SYMPTOM
1028 T4 85 88 胶囊剂 DRUG_DOSAGE
1028 T5 160 162 和皮 DRUG_INGREDIENT
1028 T6 163 166 真菌有 DRUG_INGREDIENT
1028 T7 167 170 制作用 DRUG_EFFICACY
1028 T8 207 209 勺有 DRUG_INGREDIENT
1028 T9 210 214 血通瘀去 DRUG_EFFICACY
1029 T1 0 2 孕妇 PERSON_GROUP
1029 T2 19 23 肝

1045 T1 104 108 活血调经 DRUG_EFFICACY
1045 T2 109 113 去瘀止痛 DRUG_EFFICACY
1045 T3 118 122 瘀血不净 SYMPTOM
1045 T4 132 136 瘀血不净 SYMPTOM
1045 T5 145 148 胶囊剂 DRUG_DOSAGE
1045 T6 168 172 痢?/t SYMPTOM
1045 T8 175 177 /t SYMPTOM
1045 T9 177 181 r><t DRUG_EFFICACY
1045 T9 181 185 r cl DRUG_EFFICACY
1045 T10 186 190 ss=' DRUG_EFFICACY
1045 T11 195 199 Whit SYMPTOM
1046 T1 4 8 补血健脑 DRUG_EFFICACY
1046 T2 43 46 补气血 DRUG_EFFICACY
1046 T3 58 60 头晕 SYMPTOM
1047 T1 4 8 薄膜衣片 DRUG_DOSAGE
1047 T2 23 26 气微香 DRUG_TASTE
1047 T4 102 104 宫平 PERSON_GROUP
1047 T4 104 108 滑肌有一 SYNDROME
1047 T5 113 115 ，并 SYMPTOM
1047 T6 116 118 小鼠 SYMPTOM
1047 T8 132 137 .32g  DISEASE
1047 T8 137 139  用 SYMPTOM
1047 T9 140 142 妇人 SYMPTOM
1047 T10 143 150 血阻络所所致块 SYMPTOM
1047 T11 163 165 尽； DRUG_DOSAGE
1048 T1 12 14 寒凉 FOOD_GROUP
1048 T2 15 17 生冷 FOOD_GROUP
1048 T3 41 43 藜芦 DRUG_INGREDIENT
1048 T4 44 47 五灵脂 DRUG_INGREDIENT
1048 T5 48 50 皂荚 DRUG_INGREDIENT
1048 T6 56 61 糖尿病患者 PERSON_GROUP
1048 T7 71 75 月经提前 SYMPTOM
1048 T8 81 83 感冒 DISE

1074 T1 79 80 淋 DISEASE_GROUP
1074 T2 84 88 淋巴肿大 DRUG_EFFICACY
1074 T3 89 91 疼痛 DRUG_EFFICACY
1075 T1 48 52 补阴培阳 DRUG_EFFICACY
1075 T2 53 57 益肾健脾 DRUG_EFFICACY
1075 T3 87 91 补阴培阳 DRUG_EFFICACY
1075 T4 92 97 益肾健脾  DRUG_EFFICACY
1076 T1 84 86 抗力 DRUG_INGREDIENT
1076 T2 87 89 香附 DRUG_INGREDIENT
1076 T3 90 92 延胡 DRUG_INGREDIENT
1076 T4 93 99 赤芍能抑制子 DRUG_EFFICACY
1076 T5 127 131 疗妇女月 SYMPTOM
1076 T6 132 134 不调 SYMPTOM
1076 T7 144 148 。  养 DRUG_EFFICACY
1076 T8 149 153 舒肝，调 DRUG_EFFICACY
1076 T10 156 158 。用 SYNDROME
1076 T10 158 160 于血 SYNDROME
1076 T11 162 166 郁所致月 SYMPTOM
1076 T12 167 169 不调 SYMPTOM
1076 T13 184 188 后错，经 SYMPTOM
1076 T14 189 192 量少、 SYMPTOM
1076 T15 193 199 血块，行经小 SYMPTOM
1076 T17 200 202 疼痛 SYMPTOM
1076 T17 202 204 ，血 SYMPTOM
1076 T19 303 305 医师 SYNDROME
1076 T19 305 307 或药 SYNDROME
1076 T20 309 313   每片 SYMPTOM
1076 T21 314 316 0. SYMPTOM
1077 T1 79 83 10袋  DRUG_DOSAGE
1077 T2 84 86 口服 DRUG_DOSAGE
1077 T3 95 97 次。 FOOD_GROUP
1077 T4 98 100  丸 FOOD_GROUP
1077 T5 106 109  

1103 T1 13 17 促进造血 DRUG_EFFICACY
1103 T2 18 20 止血 DRUG_EFFICACY
1103 T3 344 347 致腹腔 DRUG_DOSAGE
1103 T4 348 350 出液 DRUG_TASTE
1103 T5 351 353 白细 DRUG_TASTE
1103 T6 366 368 物（ PERSON_GROUP
1103 T7 412 414 微苦 DRUG_INGREDIENT
1103 T8 415 418   6 DRUG_INGREDIENT
1103 T9 419 421 *1 DRUG_INGREDIENT
1103 T10 448 451 本药时 DISEASE
1103 T11 452 455 宜同时 DISEASE_GROUP
1103 T12 636 640 在儿童不 SYMPTOM
1103 T14 641 647 接触的地方。 SYMPTOM
1103 T14 647 651 12.如 DRUG_EFFICACY
1103 T15 659 663 ，使用本 SYMPTOM
1103 T16 664 668 前请咨询 SYMPTOM
1103 T17 669 673 师或药师 SYMPTOM
1103 T18 674 678   6g SYMPTOM
1103 T19 679 683 12袋。 SYMPTOM
1103 T20 749 751 次6 DRUG_INGREDIENT
1103 T21 752 754 ，一 DRUG_INGREDIENT
1103 T22 755 758 2次。 DRUG_INGREDIENT
1103 T23 762 764 服用 DRUG_INGREDIENT
1103 T24 776 778 药师 DRUG_INGREDIENT
1103 T25 779 782 包括任 DRUG_INGREDIENT
1103 T26 783 785 从药 DRUG_INGREDIENT
1103 T27 786 788 、超 DRUG_INGREDIENT
1103 T28 789 791 或保 DRUG_INGREDIENT
1103 T29 795 797 购买 DRUG_INGREDIENT
1103 T30 798 800 非处 DRUG_INGREDI

...

1255 T2 55 61 收缩子宫儿童 PERSON_GROUP
1255 T2 61 63 孕妇 PERSON_GROUP
1255 T4 89 91 儿童 PERSON_GROUP
1255 T5 91 93 儿童 PERSON_GROUP
1255 T5 93 95 孕妇 PERSON_GROUP
1255 T6 107 111 排出瘀血 DRUG_EFFICACY
1255 T7 302 306 降低血液 DRUG_EFFICACY
1255 T8 326 329 易凝固 DRUG_EFFICACY
1255 T9 504 513 循环：增加子宫毛细 DRUG_EFFICACY
1255 T10 514 516 管流 DRUG_EFFICACY
1255 T11 524 528  活血、 SYMPTOM
1255 T12 531 535 止痛。用 SYMPTOM
1255 T13 541 545 行，症见 SYMPTOM
1255 T14 558 562 冷。也可 SYMPTOM
1255 T16 563 567 用于上节 SYMPTOM
1255 T17 567 571 育环引起 SYMPTOM
1255 T17 571 575 的阴道流 SYMPTOM
1255 T18 576 582 ，月经过多形 SYMPTOM
1255 T19 583 585 肢冷 DRUG_EFFICACY
1255 T20 586 588 腹冷 DRUG_EFFICACY
1255 T21 593 597 按活血、 SYMPTOM
1255 T22 623 625 血管 DRUG_EFFICACY
1255 T23 710 714 医保目录 SYMPTOM
1255 T24 732 736 痛，也可 SYMPTOM
1255 T25 737 742 于上节育环 SYMPTOM
1255 T26 929 933 明显减少 DRUG_EFFICACY
1255 T28 982 985 液流变 DRUG_EFFICACY
1255 T28 985 991 学表明，新生 DRUG_EFFICACY
1255 T29 1069 1073 生化颗粒 DRUG_EFFICACY
1255 T30 1093 1096 及抗血 DRUG_EFFICACY
1255 T31 1280 1282 

1274 T1 8 10 孕妇 PERSON_GROUP
1274 T2 69 71 滞、 DRUG_INGREDIENT
1274 T3 73 77 虚寒所致 DRUG_EFFICACY
1274 T4 78 80 月经 DRUG_EFFICACY
1274 T5 117 119 缩， DRUG_INGREDIENT
1274 T6 121 125 其紧张度 DRUG_EFFICACY
1274 T7 126 128 功能 DRUG_EFFICACY
1274 T8 142 144 溶、 DRUG_EFFICACY
1274 T9 153 159 附镇痛、抑制 DRUG_EFFICACY
1274 T10 195 199 作用。3 SYMPTOM
1274 T11 200 202 香附 SYMPTOM
1274 T12 203 207 提高痛阈 SYMPTOM
1274 T13 208 210 镇痛 SYMPTOM
1274 T14 211 213 松弛 SYMPTOM
1274 T15 217 219 肌， SYNDROME
1274 T16 230 237 张度和抑制金黄 SYMPTOM
1274 T17 252 254 有雌 SYMPTOM
1274 T18 255 259 素样作用 SYMPTOM
1274 T19 260 262 4. SYMPTOM
1274 T20 314 316 性， SYMPTOM
1274 T21 317 321 抗乙酰胆 SYMPTOM
1274 T22 322 324 的作 SYMPTOM
1274 T23 383 387 碱的作用 DRUG_EFFICACY
1274 T25 390 392 用治 SYNDROME
1274 T25 392 394 胃痛 SYNDROME
1274 T26 407 409 定的 SYMPTOM
1274 T27 412 416   孕妇 SYMPTOM
1274 T28 417 420 用。  SYMPTOM
1274 T29 421 424 如与其 SYMPTOM
1274 T30 425 429 药物同时 SYMPTOM
1274 T31 430 438 用可能会发生药物 SYMPTOM
1275 T1 13 15    DRUG_DOSAGE
1275 T2 39 43 疏肝活血 D

1296 T2 57 59 血虚 SYNDROME
1296 T2 59 61 气滞 SYNDROME
1296 T3 69 73 月经不调 SYMPTOM
1296 T4 125 129 暖宫调经 DRUG_EFFICACY
1296 T6 132 134 血虚 SYNDROME
1296 T6 134 136 气滞 SYNDROME
1296 T7 149 151 痛经 SYMPTOM
1296 T8 154 158 行经后错 SYMPTOM
1296 T9 159 162 经量少 SYMPTOM
1296 T10 163 166 有血块 SYMPTOM
1296 T11 167 171 小腹疼痛 SYMPTOM
1296 T12 172 180 经行小腹冷痛喜热 SYMPTOM
1296 T13 217 219 明， DRUG_EFFICACY
1296 T14 236 238 滑肌 DRUG_INGREDIENT
1296 T15 240 244 ，松弛其 DRUG_EFFICACY
1296 T16 245 247 张度 DRUG_EFFICACY
1296 T17 261 263 抑制 DRUG_EFFICACY
1296 T18 272 278 3.香附能提 DRUG_EFFICACY
1296 T19 314 318 球菌，某 SYMPTOM
1296 T20 319 321 真菌 SYMPTOM
1296 T21 322 326 长，并具 SYMPTOM
1296 T22 327 329 雌激 SYMPTOM
1296 T23 330 332 样作 SYMPTOM
1296 T24 336 338 故临 SYNDROME
1296 T25 349 356 不调，痛经，宫 SYMPTOM
1296 T26 371 373 证型 SYMPTOM
1296 T27 374 378 。此外， SYMPTOM
1296 T28 379 381 丸还 SYMPTOM
1296 T29 404 408 胃痛，慢 DRUG_DOSAGE
1296 T30 409 412 肠炎， DRUG_DOSAGE
1296 T31 415 417 病证 PERSON_GROUP
1297 T1 8 11 补气血 DRUG_EFFICACY
1297 T2 12 15 调月经 

1317 T1 39 43 月经不调 SYMPTOM
1317 T2 46 52 月经周期错后 SYMPTOM
1317 T3 53 57 行经量少 SYMPTOM
1317 T4 58 62 精神不振 SYMPTOM
1317 T5 63 67 肢体乏力 SYMPTOM
1317 T6 73 77 泰药业股 SYNDROME
1317 T7 91 97 经。用于气血 SYMPTOM
1317 T8 98 102 虚兼有血 SYMPTOM
1317 T9 103 107 所致的月 SYMPTOM
1317 T10 108 112 不调。症 SYMPTOM
1317 T11 150 153 ）,国 DRUG_EFFICACY
1318 T1 36 40 月经不调 SYMPTOM
1318 T2 43 49 月经周期错后 SYMPTOM
1318 T3 108 112 口服。一 DRUG_EFFICACY
1318 T4 115 119 ，一日2 SYNDROME
1318 T5 133 139 远药业有限公 SYMPTOM
1318 T6 140 144   补气 SYMPTOM
1318 T7 145 149 ，调月经 SYMPTOM
1318 T8 150 154   益气 SYMPTOM
1319 T2 38 40 血虚 SYNDROME
1319 T2 40 42 肝郁 SYNDROME
1319 T3 49 51 痛经 SYMPTOM
1319 T4 77 79 孕妇 PERSON_GROUP
1319 T5 102 104 辛辣 FOOD_GROUP
1319 T6 105 107 生冷 FOOD_GROUP
1319 T7 112 118 感冒发热病人 PERSON_GROUP
1319 T8 126 129 高血压 DISEASE
1319 T9 130 133 心脏病 DISEASE_GROUP
1319 T10 134 136 肝病 DISEASE_GROUP
1319 T11 137 140 糖尿病 DISEASE
1319 T12 141 143 肾病 DISEASE_GROUP
1319 T13 144 147 慢性病 DISEASE_GROUP
1319 T14 162 167 青春期少女 PERSON_GROUP
1319 T15 168

1340 T1 41 43 人参 DRUG_INGREDIENT
1340 T2 44 46 白芍 DRUG_INGREDIENT
1340 T3 47 50 反藜芦 DRUG_INGREDIENT
1340 T4 54 56 藜芦 DRUG_INGREDIENT
1340 T5 68 70 甘草 DRUG_INGREDIENT
1340 T6 71 74 反甘遂 DRUG_INGREDIENT
1340 T7 75 77 大戟 DRUG_INGREDIENT
1340 T8 78 80 海藻 DRUG_INGREDIENT
1340 T9 81 83 芫花 DRUG_INGREDIENT
1340 T10 87 89 甘遂 DRUG_INGREDIENT
1340 T11 90 92 大戟 DRUG_INGREDIENT
1340 T12 93 95 海藻 DRUG_INGREDIENT
1340 T13 96 98 芫花 DRUG_INGREDIENT
1340 T14 111 113 生冷 FOOD_GROUP
1340 T15 114 116 辛辣 FOOD_GROUP
1340 T17 117 119 荤腥 FOOD_GROUP
1340 T17 119 121 油腻 FOOD_GROUP
1340 T18 122 126 不易消化 FOOD_GROUP
1340 T19 166 169 五灵脂 DRUG_INGREDIENT
1340 T20 170 172 皂荚 DRUG_INGREDIENT
1340 T21 216 220 促进造血 DRUG_EFFICACY
1340 T22 221 223 止血 DRUG_EFFICACY
1340 T23 563 567 如果服用 DRUG_EFFICACY
1340 T24 568 572 何其他药 DRUG_EFFICACY
1340 T25 575 579 知医师或 SYNDROME
1340 T26 580 584 师，包括 SYMPTOM
1340 T27 585 589 何从药房 SYMPTOM
1340 T29 590 594 超市或保 SYMPTOM
1340 T29 594 596 健品 PERSON_GROUP
1340 T30 668 670 国家 PERSON_GROUP
1341 T

In [ ]:
result

In [24]:
from keras.models import load_model
import numpy as np

maxlen = 40
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"
#sentence = sentence.replace(' ','_')
sentences = sentence.split('。')
y_ner = []

for sent in sentences:
    sent = sent.replace(' ','_')
    sent_chars = list(sent+'。')
    sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

    sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
    y_pred = model.predict(sent2id_new)
    y_label = np.argmax(y_pred, axis=2)
    y_label = y_label.reshape(1, -1)[0]
    y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
    y_ner.extend(y_ner_)

In [27]:
result_words = get_valid_nertag(sentence, y_ner)
for (word, tag) in result_words:
    print("".join(word), tag)

感冒 DISEASE
发热 SYMPTOM
高血压 DISEASE
心脏病 DISEASE_GROUP
肝病 DISEASE_GROUP
糖尿病 DISEASE
肾病 DISEASE_GROUP
慢性病 DISEASE_GROUP
月经紊乱 SYMPTOM
眩晕 SYMPTOM
过敏者 PERSON_GROUP
儿童 PERSON_GROUP
颗粒 DRUG_DOSAGE
气微香 DRUG_TASTE
味微苦 DRUG_TASTE
  滋养肝肾 DRUG_EFFICACY
宁心安神 DRUG_EFFICACY
烘热汗出 SYMPTOM
头晕耳鸣 SYMPTOM
失眠多梦 SYMPTOM
五心烦热 SYMPTOM
腰背酸 SYMPTOM


In [25]:
list(zip(list(sentence),y_ner))

[(' ', 'O'),
 ('北', 'O'),
 ('京', 'O'),
 ('同', 'O'),
 ('仁', 'O'),
 ('堂', 'O'),
 ('科', 'O'),
 ('技', 'O'),
 ('发', 'O'),
 ('展', 'O'),
 ('股', 'O'),
 ('份', 'O'),
 ('有', 'O'),
 ('限', 'O'),
 ('公', 'O'),
 ('司', 'O'),
 ('制', 'O'),
 ('药', 'O'),
 ('厂', 'O'),
 (' ', 'O'),
 (' ', 'O'),
 ('1', 'O'),
 ('.', 'O'),
 ('忌', 'O'),
 ('食', 'O'),
 ('辛', 'O'),
 ('辣', 'O'),
 ('，', 'O'),
 ('少', 'O'),
 ('进', 'O'),
 ('油', 'O'),
 ('腻', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('2', 'O'),
 ('.', 'O'),
 ('感', 'B-DISEASE'),
 ('冒', 'I-DISEASE'),
 ('发', 'B-SYMPTOM'),
 ('热', 'I-SYMPTOM'),
 ('病', 'O'),
 ('人', 'O'),
 ('不', 'O'),
 ('宜', 'O'),
 ('服', 'O'),
 ('用', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('3', 'O'),
 ('.', 'O'),
 ('有', 'O'),
 ('高', 'B-DISEASE'),
 ('血', 'I-DISEASE'),
 ('压', 'I-DISEASE'),
 ('、', 'O'),
 ('心', 'B-DISEASE_GROUP'),
 ('脏', 'I-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('肝', 'B-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('糖', 'B-DISEASE'),
 ('尿', 'I-DISEASE'),
 ('病', 'I-DISEASE'),
 ('、'